Leonardo’s centralized church
=============================

Aim of this workshop is to provide a personally interpreted PyPLaSM model for one of Leonardo Da Vinci's centalized churches. These kind of churches don't have a nave and show a circular or star-shaped symmetry. This architectual approach has similarities with the great Duomo in Florence, the Santa Maria del Fiore, which was a familiar sight to Leonardo.

In [2]:
from pyplasm import *

Evaluating fenvs.py..
...fenvs.py imported in 0.009474 seconds


## Base
Base of the church consists of a circular staircase made of three steps.

In [3]:
def base(diameter):
    """
    base(10)
    :param diameter: diameter of the bottom step
    :return: pyplasm.xgepy.Hpc
    """
    stepBottom = CYLINDER([diameter+0.5, 0.3])(40)
    stepMiddle = T(3)(0.3)(CYLINDER([diameter, 0.3])(40))
    stepTop = T(3)(0.6)(CYLINDER([diameter-0.5, 0.3])(40))
    return STRUCT([stepBottom, stepMiddle, stepTop])

In [6]:
b = base(10)
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10802bf30> >

![staircase](./images/staircase.png)

## Dome
The dome is a semisphere. Pyplasm.SPHERE has been customized to obtain a SEMISPHERE function 

In [4]:
def SEMISPHERE (radius):
    """
    SEMISPHERE(10)([8,20]) 
    :param radius: radius of the SEMISPHERE
    :param subds: subcells forming the (semi)sphere
    :return: pyplasm.xgepy.Hpc
    """
    def SPHERE0 (subds):
        N , M = subds
        domain = Plasm.translate( Plasm.power(INTERVALS(PI/2)(N) , INTERVALS(2*PI)(M)), Vecf(0, -PI/2,0 ) )
        fx  = lambda p: radius * math.cos(p[0])  * math.sin  (p[1])
        fy  = lambda p: radius * math.cos(p[0]) * math.cos (p[1])
        fz  = lambda p: radius * math.sin(p[0])
        ret=  MAP([fx, fy, fz])(domain)
        return ret
    return SPHERE0

def dome(radius):
    """
    dome(10)
    :param radius: radius of the dome
    :return: pyplasm.xgepy.Hpc
    """
    d = JOIN([SEMISPHERE(radius)([8,20])])
    return R([1,3])(-PI)(d)

In [10]:
myDome = dome(10)
VIEW(myDome)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10d3809f0> >

![dome](./images/dome.png)

## Walls
Thick circular walls. The simplest walls have neither absides or naves.


In [5]:
def wallsFULL(diameter, height):
    """
    wallsFULL(10,8)
    :param diameter: diameter for the outer surface of the walls
    :param height: height of the walls
    :return: pyplasm.xgepy.Hpc
    """
    outside = CYLINDER([diameter, height])(60)
    inside = CYLINDER([diameter-4.5, height])(60)
    w = DIFFERENCE([outside, inside])
    return STRUCT([w])

In [6]:
VIEW(wallsFULL(10,8))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10b9abc90> >

![full walls](./images/fullwalls.png)

## Apsis 
" In architecture, an apse (plural apses; from Latin absis: "arch, vault" from Greek ἀψίς apsis "arch"; sometimes written apsis, plural apsides) is a semicircular recess covered with a hemispherical vault or semi-dome, also known as an Exedra. In Byzantine, Romanesque, and Gothic Christian church (including cathedral and abbey) architecture, the term is applied to a semi-circular or polygonal termination of the main building at the liturgical east end (where the altar is), regardless of the shape of the roof, which may be flat, sloping, domed, or hemispherical. Smaller apses may also be in other locations, especially shrines. " (Wikipedia.org)

In [8]:
def apsis(wallThinckness, heightWalls):
    """
    apsis(4, 8)
    :param wallThinckness: thickness of the walls as the difference of two cylinders
    :param height: height of the walls
    :return: pyplasm.xgepy.Hpc
    
      __^^^^^__
      |   2    |
      |        |
    <            >
    < 1        3 >
    <            >
      |        |
      |__  4  _|
         vvvv
    """
    central = T([1,2])([-1.5,-1.5])(CUBOID([3,3,heightWalls]))
    cylinder1 = T(1)(-1)(CYLINDER([1,heightWalls])(40))
    cylinder2 = T(2)(-1)(CYLINDER([1,heightWalls])(40))
    cylinder3 = T(1)(+1)(CYLINDER([1,heightWalls])(40))
    cylinder4 = T(2)(+1)(CYLINDER([1,heightWalls])(40))

    sphericalCap = T(3)(heightWalls)(dome(1.5))
    return STRUCT([central, cylinder1, cylinder2, cylinder3, cylinder4, sphericalCap])

In [9]:
VIEW(apsis(4,8))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1065ac030> >

![apsis](./images/apsis_1.png)
![apsis](./images/apsis_2.png)

### absides inside the walls

In [11]:
def placeCircular(positions, rad, objects, entrance, offset):
    """
    placeCircular(8, 5, objectsList, 1, 0)
    :param positions: number of equidistant object to place
    :param rad: radius from the center of the church where the objects are placed forming a circumference
    :param objects: array of objects (ex. absides oriented toward the center of the church)
    :param entrance: leaves space for the entrance (VALUES: 1/0).
    :param offset: external recesses are offsetted of +PI/recesses
    :return: pyplasm.xgepy.Hpc
    """
    return map(lambda i: T([1, 2])([rad*math.cos((2*PI/positions)*(i+entrance) +offset), rad*math.sin((2*PI/positions)*(i+entrance) + offset)])(R([1,2])((2*PI/positions)*i)(objects[i])), range(0, positions-entrance))


In [11]:
def placeAbsides(nAbsides, diameter, thicknessWalls, heightWalls):
    """
    placeAbsides(8, 10, 4, 8)
    :param nAbsides: 
    :param diameter: 
    :param thicknessWalls: 
    :param heightWalls: 
    :return: 
    """
    r = diameter-diameter/6 - 2
    ab = [apsis(thicknessWalls, heightWalls) for i in range(0, nAbsides)]
    return STRUCT(placeCircular(nAbsides, r, ab, 1, 0))

In [13]:
VIEW(placeAbsides(8, 10, 4, 8))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1065acd50> >

![absides](./images/absides.png)

## Entrance
Let's create space for the entrance

In [17]:
def cutEntrance(dimension):
    """
    cutEntrance(9)
    :param dimension: dimension of the area to be cut from the walls 
    :return: pyplasm.xgepy.Hpc
    """
    points = [[0,0,0], [dimension,1.5,0], [dimension, -1.5, 0], [dimension,1.5,6], [dimension, -1.5, 6], [0,0,0]] # 1.5 = half the apsis dimension
    toBRemoved = JOIN(AA(MK)(points))
    return STRUCT([toBRemoved])

In [19]:
VIEW(cutEntrance(9))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10d7a36f0> >

## Final walls
Let's put all togheter. Walls + absides + recesses

In [20]:
def structuredWalls(nAbsides, diameter, height, thickness):
    """
    structuredWalls(7, 9, 8, 4)
    :param nAbsides: number of absides
    :param diameter: diameter of the walls
    :param height: height of the walls
    :param thickness: thickness of the walls
    :return: pyplasm.xgepy.Hpc
    """
    #absides: 0 radiusOffset, 1 thicknessCoeff (unchanged), 0 circularOffset
    absides = placeAbsides(nAbsides, diameter, thickness, height+2)
    entranceSpace = cutEntrance(diameter)

    #these objects will provide the pattern for the recesses in the walls
    recessObjects = [CYLINDER([diameter/8,height])(8) for i in range(0, nAbsides)]
    r = diameter-diameter/6 + 1
    recess = STRUCT(placeCircular(nAbsides, r, recessObjects, 0, PI/nAbsides))

    removableSpace = STRUCT([absides, entranceSpace, recess])

    walls = wallsFULL(diameter, height)
    return DIFFERENCE([walls, removableSpace])

In [21]:
VIEW(structuredWalls(7, 9, 8, 4))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10d7a36c0> >

![walls](./images/placeabsides.png)
![walls](./images/placeabsidesrecesses.png)

## Top of the structure
The top of this structure consists in a group of minor domes surrounding a bigger central one

In [26]:
def subDome(radius, height, colorHexDome, colorHexCylinder):
    """
    subDome(2, 2, "#841F27", "#E2DECD")
    :param radius: radius of the dome
    :param height: height of the main structure of the church of which the subdome is proportioned to
    :param colorHexDome: dome color
    :param colorHexCylinder:  basis of the dome color
    :return: pyplasm.xgepy.Hpc
    """
    sBottom1 = CYLINDER([radius+0.3, height*0.15])(50)
    sBottom2 = T(3)(height*0.15)(CYLINDER([radius+0.2, height*0.15])(50))
    sCylinder = T(3)(height*0.3)(CYLINDER([radius-0.1, height])(10))
    sTop = T(3)(height*1.3)(CYLINDER([radius+0.3, height*0.10])(50))
    sDome = T(3)(height*1.4)(dome(radius))

    tipBottom = T([1, 2, 3])([-height*0.05, -height*0.05, height*1.4+radius])(CUBOID([height*0.1]*3))
    tipMiddle = T([1, 2, 3])([-height*0.05, -height*0.05, height*1.5+radius])(CUBOID([height*0.1, height*0.1, height*0.3]))
    tipTop1 = T(3)(height*1.8+radius)(SEMISPHERE(height*0.1)([8,20]))
    tipTop2 = T(3)(height*1.9+radius)(SPHERE(height*0.1)([8,20]))

    coloredDome = HEX(colorHexDome)(STRUCT([sDome, tipTop1, tipTop2]))
    coloredCylinder =  HEX(colorHexCylinder)(STRUCT([sBottom1, sBottom2, sCylinder, sTop, tipBottom, tipMiddle])) 
    return STRUCT([coloredDome, coloredCylinder])

In [24]:
VIEW(subDome(2, 2, "#841F27", "#E2DECD"))

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10d7a3c00> >

![subdome](./images/subdome.png)

In [31]:
def placeSubDomes(nSubDomes, diameterStruct, size, colorHexDome, colorHexCylinder):
    """
    placeSubDomes(8, 10, 2, "#841F27", "#E2DECD")
    :param nSubDomes: number of subdomes
    :param diameterStruct: diameter of the main structure
    :param size: size of the subdome
    :param colorHexDome: dome color
    :param colorHexCylinder:  basis of the dome color
    :return: pyplasm.xgepy.Hpc
    """
    r = diameterStruct*0.72
    sd = [(subDome(size, size, colorHexDome, colorHexCylinder)) for i in range(nSubDomes)]
    return STRUCT(placeCircular(nSubDomes, r, sd, 0, 0))

In [32]:
VIEW(placeSubDomes(8, 10, 2, "#841F27", "#E2DECD"))

Child 3
Child 3
Child

 3
Child 3
Child 3


Child 3


Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1065cf510> >

![subdome](./images/subdomes.png)

In [34]:
## a simple square outside the church

In [35]:

def disk2D(p):
    u,v = p
    return [v*COS(u), v*SIN(u)]

domain2D = PROD([INTERVALS(2*PI)(20), INTERVALS(10)(3)]) # 2D domain decompos VIEW( MAP(disk2D)(domain2D) )

def square(p):
    return STRUCT([PROD([MAP(disk2D)(domain2D),Q(0.5)]), HEX("#841F27")(SKELETON(1)(PROD([MAP(disk2D)(domain2D),Q(0.5)])))])

In [36]:
VIEW(square(10))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10587d030> >

![square](./images/square.png)

#Leonardo's church
Putting all togheter to create the entire model

In [37]:
def leonardo():
    diameter = 10
    height = 8
    travertine = "#E2DECD"
    brick = "#841F27"

    #green grass
    leoGround = T([1,2])([-30, -30])(MATERIAL([0,0,0,1,  0,.1,0,1,  0,.1,0,1, 0,0,0,1, 0])(CUBOID([90, 60, 0.2])))

    # 3 steps, the base for the main structure, translated on the 3rd axis beacause of the ground
    leoBase = T(3)(0.2)(base(diameter))

    # a simple square outside the church
    leoPath2Square = T([1,2,3])([diameter+0.2, -2, 0.2])(CUBOID([diameter*2, 4, 0.4]))
    leoSquare = T([1,3])([diameter*3, 0.2])(square(diameter))

    #steps + grounds
    leoWalls = T(3)(0.3*3 + 0.2)(structuredWalls(7, diameter-1, height, 4))

    verts = CIRCLE_POINTS(diameter+0.2,50)
    # T(3) -> main structure of the church + steps size + ground
    leoTop = T(3)(height + 0.3*3 + 0.2)(PROD([JOIN(AA(MK)(verts)), Q(0.5)]))

    leoSubDomes = T(3)(height + 0.3*3 + 0.5 + 0.2)(placeSubDomes(nSubDomes=8, diameterStruct=diameter, size=diameter/4, colorHexDome=brick, colorHexCylinder=travertine))

    leoBiggerDome = T(3)(height + 0.3*3 + 0.5 + 0.2)(subDome(diameter*0.45, height*0.7, brick, travertine))

    travertineStructure = HEX(travertine)(STRUCT([leoGround, leoBase, leoWalls, leoTop, leoPath2Square, leoSquare]))
    return STRUCT([travertineStructure, leoSubDomes, leoBiggerDome])


In [38]:
VIEW(leonardo())

Child 3
Child 3
Child 3


Child 3
Child 3


Child 3
Child 3


Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1065d51e0> >

![leonardo1](./images/leonardo_1.png)
![leonardo2](./images/leonardo_2.png)
![leonardo3](./images/leonardo_3.png)
![leonardo4](./images/leonardo_4.png)